In [1]:
# This is a minimal reproducible example for the state "AK".

In [1]:
# Load libraries
import os
import pandas as pd
import geopandas as gpd
import requests
import gzip
import json
from shapely.geometry import shape
from tqdm import tqdm
import numpy as np
import math
import zipfile
import fiona
import zipfile

In [2]:
# Download datasets and needed files

def download_and_save(url, dest_path):
    """Download a file from a given URL and save it locally."""
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    with open(dest_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192): 
            file.write(chunk)

# Extract state code from the ZIP URL
zip_url = 'https://disasters.geoplatform.gov/publicdata/Partners/ORNL/USA_Structures/Alaska/Deliverable20230728AK.zip'
state_code = zip_url.split('Deliverable')[1][8:10]  # Extracts 'AK' from the provided URL

# Define directories
base_dir = './downloads'
fema_dir = os.path.join(base_dir, 'FEMA')
state_folder = os.path.join(fema_dir, state_code)
zip_folder = os.path.join(state_folder, 'zip_files')
extract_folder = os.path.join(state_folder, 'extracted_files')
geojson_folder = os.path.join(base_dir, 'geojson_files')
csv_folder = os.path.join(base_dir, 'csv_files')

geojson_url = 'https://gist.githubusercontent.com/Feng96/bf831f5cf972d24455a25128ed8b3636/raw/f05772ff636c790f6ebc0e4eb5aa6f7b669f4ea9/data_link.geojsonn'
csv_url = 'https://minedbuildings.blob.core.windows.net/global-buildings/dataset-links.csv'

# Create directories
os.makedirs(zip_folder, exist_ok=True)
os.makedirs(extract_folder, exist_ok=True)
os.makedirs(geojson_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)

zip_dest_path = os.path.join(zip_folder, f'Deliverable{state_code}.zip')
geojson_dest_path = os.path.join(geojson_folder, 'code.geojson')
csv_dest_path = os.path.join(csv_folder, 'dataset-links.csv')

# Download and save files
download_and_save(zip_url, zip_dest_path) # FEMA US Structures ZIP file
download_and_save(geojson_url, geojson_dest_path) # MS buildings with height coverage GeoJSON file
download_and_save(csv_url, csv_dest_path) # CSV file with links to the MS footprint data

# Extract ZIP content
with zipfile.ZipFile(zip_dest_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)


print('FEMA and MS data downloaded and extracted successfully!')


FEMA and MS data downloaded and extracted successfully!


In [3]:
df = pd.read_csv(os.path.join(csv_folder, 'dataset_link.csv'))

gdf = gpd.read_file(os.path.join(geojson_folder, 'code.geojson'))

In [ ]:
states_url = 'https://raw.githubusercontent.com/giswqs/data/main/us/us_states.csv'
states = pd.read_csv(states_url)
print(states.head())

In [9]:
#loop through each state
state_name = "Alaska"
state = state_code
gdf_state = gdf[gdf['STUSPS'] == state].copy()  # Create a copy

print(f"Processing state: {state_name} ({state})")

# Convert 'field_1' to 'QuadKey' as a string
gdf_state.loc[:, 'QuadKey'] = gdf_state['field_1'].apply(lambda x: str(int(str(x)[1:9])))

# Ensure 'QuadKey' in df is also a string
df['QuadKey'] = df['QuadKey'].astype(str)

# Filter out rows where 'QuadKey' does not match with 'QuadKey' in gdf_state
df_state = df[df['QuadKey'].isin(gdf_state['QuadKey'])]

print(f"Downloading files for state: {state_name} ({state})")
# Create directory to save downloaded files
os.makedirs(os.path.join(base_dir, f'downloaded_files/{state}'), exist_ok=True)

# Iterate over all rows in the filtered dataframe
for idx, row in tqdm(df_state.iterrows(), total=df_state.shape[0]):
        # Get the URL
    url = row['Url']

    # Make a GET request to the URL
    response = requests.get(url, stream=True)

    # Check if the request was successful
    if response.status_code == 200:
        # Get the file name by joining the 'downloaded_files' with the basename of the URL
        file_name = os.path.join(base_dir, f'downloaded_files/{state}', f"{state}_{idx}.csv.gz")

        # Open the file in write mode
        with open(file_name, 'wb') as file:
            # Write the content of the request to the file
            for chunk in response.iter_content(chunk_size=128):
                file.write(chunk)
    else:
        print(f"Failed to download file from url {url}")

print(f"Processing files for state: {state_name} ({state})")

gdf_list = []
# Iterate over each .csv.gz file in the directory

download_dir = os.path.join(base_dir, 'downloaded_files', state)

for filename in tqdm(os.listdir(os.path.join(base_dir, 'downloaded_files', state))):
    if filename.endswith('.csv.gz'):
        # Join the directory with the filename
        file_path = os.path.join(base_dir, 'downloaded_files', state, filename)

        # Open the .csv.gz file
        with gzip.open(file_path, 'rt') as file:
            # Read each line in the .csv.gz file
            for line in file:
                # Remove leading/trailing whitespace
                line = line.strip()

                # Skip empty lines
                if not line:
                        continue

                # Parse the line as JSON
                data = json.loads(line)

                # Create a GeoDataFrame from the data and append it to the list
                geometry = shape(data['geometry'])
                properties = data['properties']
                data_gdf = gpd.GeoDataFrame([properties], geometry=[geometry])
                gdf_list.append(data_gdf)

# Concatenate all the GeoDataFrames in the list into a single GeoDataFrame
final_gdf = pd.concat(gdf_list, ignore_index=True)
final_gdf['height'] = final_gdf['height'].astype(float).round(2)
# Keep only 'height' attribute and geometry
final_gdf = final_gdf[['height', 'geometry']]

# Load both GeoDataFrames

# Load the shapefile
gdf2_path = os.path.join(extract_folder, f"Deliverable20230728AK", f"{state_code}_Structures.gdb")
layers = fiona.listlayers(gdf2_path)
print(f"Available layers: {layers}")
gdf2 = gpd.read_file(gdf2_path, layer=layers[0])
    

final_gdf = final_gdf.set_crs('EPSG:4326')
gdf2 = gdf2.set_crs('EPSG:4326')


gdf2['HEIGHT'] = gdf2['HEIGHT'].astype(float).round(2)
gdf2 = gdf2[['geometry', 'HEIGHT']]

final_gdf['height'] = final_gdf['height'].apply(lambda x: np.nan if x <= 0 else x)
gdf2['HEIGHT'] = gdf2['HEIGHT'].apply(lambda x: np.nan if x == 0 else x)

# Perform spatial indexing on both GeoDataFrames
final_gdf.sindex
gdf2.sindex

# Perform the spatial join
joined = gpd.sjoin(final_gdf, gdf2, how='left', op='intersects')

# Calculate 'diff' and 'mean'
joined['diff'] = joined.apply(lambda row: row['height'] - row['HEIGHT'] if pd.notnull(row['height']) and pd.notnull(row['HEIGHT']) else np.nan, axis=1)
joined['mean'] = joined.apply(lambda row: np.mean([val for val in [row['height'], row['HEIGHT']] if pd.notnull(val)]) if pd.notnull(row['height']) or pd.notnull(row['HEIGHT']) else np.nan, axis=1)

joined = joined.drop(columns=['index_right'])

joined = joined.to_crs('EPSG:5070')

joined['SQMETERS'] = joined['geometry'].area
joined['SQMETERS'] = joined['SQMETERS'].round(2)

joined = joined.to_crs('EPSG:4326')

joined = joined.rename(columns={'height': 'height_MS'})
joined = joined.rename(columns={'HEIGHT': 'height_FM'})
joined = joined.rename(columns={'diff': 'height_dif'})
joined = joined.rename(columns={'mean': 'height_avg'})

schema = {'geometry': 'Polygon',
'properties': {'height_MS': 'float:10.2',
            'height_FM': 'float:10.2',
            'height_dif': 'float:10.2',
            'height_avg': 'float:10.2',
            'SQMETERS': 'float:15.2',}}

# Export to GeoJSON
print(f"Saving shp file for state: {state_name} ({state})")
output_filename = os.path.join(base_dir, 'joined', f"{state}.gpkg")
joined.to_file(output_filename,driver="GPKG",schema=schema)


print("Processing complete.")

KeyError: 'STUSPS'